In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
import joblib


# تحميل البيانات
df = pd.read_csv("dataset.csv")

# حذف الصفوف التي تحتوي على NaN في المخرجات
df = df.dropna(subset=[
    'predicted_disease', 'predicted_risk_level',
    'recommended_vaccine', 'preventive_measures',
    'disease_probability_next_week'
])

# تحديد المخرجات
y_class = df[['predicted_disease', 'predicted_risk_level', 'recommended_vaccine', 'preventive_measures']]
y_reg = df['disease_probability_next_week']

# تحديد الميزات (X)
X = df.drop(columns=y_class.columns.tolist() + ['disease_probability_next_week'])

# تشفير الأعمدة النصية
label_encoders = {}
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# مقياس الخصائص
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# تقسيم البيانات
from sklearn.model_selection import train_test_split
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X_scaled, y_class, y_reg, test_size=0.2, random_state=42
)

# تدريب النماذج
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42))
classifier.fit(X_train, y_class_train)

regressor = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
regressor.fit(X_train, y_reg_train.values.reshape(-1, 1))


MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=42))

In [70]:
# التنبؤ بالتصنيفات
y_class_pred = classifier.predict(X_test)

# التنبؤ بالاحتمالية
y_reg_pred = regressor.predict(X_test)



In [72]:
from sklearn.metrics import classification_report, mean_squared_error

# توقعات نموذج التصنيف
y_class_pred = classifier.predict(X_test)

# توقعات نموذج الانحدار
y_reg_pred = regressor.predict(X_test)

print("📊 تقييم نموذج التصنيف:")
for i, col in enumerate(y_class.columns):
    print(f"--- تقييم العمود: {col} ---")
    print(classification_report(y_class_test[col], y_class_pred[:, i], zero_division=0))

print("📈 تقييم نموذج الاحتمالية (الانحدار):")
mse = mean_squared_error(y_reg_test, y_reg_pred)
print(f"Mean Squared Error: {mse}")


📊 تقييم نموذج التصنيف:
--- تقييم العمود: predicted_disease ---
              precision    recall  f1-score   support

     Allergy       0.00      0.00      0.00         1
      Asthma       0.00      0.00      0.00         3
    COVID-19       0.60      1.00      0.75         3
         Flu       0.43      1.00      0.60         3
  Heatstroke       1.00      1.00      1.00         1
   Pneumonia       0.00      0.00      0.00         2

    accuracy                           0.54        13
   macro avg       0.34      0.50      0.39        13
weighted avg       0.31      0.54      0.39        13

--- تقييم العمود: predicted_risk_level ---
              precision    recall  f1-score   support

    Critical       0.00      0.00      0.00         3
        High       0.33      1.00      0.50         2
         Low       0.25      1.00      0.40         1
      Medium       1.00      0.43      0.60         7

    accuracy                           0.46        13
   macro avg       0.40  

In [74]:
import joblib

# حفظ الموديل
joblib.dump(classifier, 'Healthon/classifier_model.pkl')
joblib.dump(regressor, 'Healthon/regressor_model.pkl')

# حفظ الـ LabelEncoders و StandardScaler بنفس الطريقة لو أنت تستخدمهم في التنبؤ
joblib.dump(label_encoders, 'Healthon/label_encoders.pkl')
joblib.dump(scaler, 'Healthon/scaler.pkl')


['Healthon/scaler.pkl']